<a href="https://colab.research.google.com/github/Alexandramejia/artificial-intelligence-foundations/blob/main/Homework02_IntelligentAgent.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Intelligent Agent

An **agent** is anything that can be viewed as perceiving its environment through sensors and
acting upon that environment. Let's create an intelligent agent that can navigate through a grid world to reach a target while avoiding obstacles.

### Task 1: Environment Setup


In [207]:
# Use a 2D list to represent the grid.
list2D = [["O", "_", "_", "_", "_"],
          ["_", "X", "_", "_", "_"],
          ["_", "_", "_", "X", "_"],
          ["_", "_", "_", "_", "_"],
          ["_", "_", "_", "_", "T"]]

In [208]:
# Printing the list directly is not very helpful for visualization
print(list2D)

[['O', '_', '_', '_', '_'], ['_', 'X', '_', '_', '_'], ['_', '_', '_', 'X', '_'], ['_', '_', '_', '_', '_'], ['_', '_', '_', '_', 'T']]


In [209]:
# Create a print function to better present the grid
def print_grid(list2D):
    for i in range(len(list2D)):
        for j in range(len(list2D[0])):
            print(list2D[i][j],end="  ")
        print()

In [210]:
print_grid(list2D)

O  _  _  _  _  
_  X  _  _  _  
_  _  _  X  _  
_  _  _  _  _  
_  _  _  _  T  


In [211]:
# We can use random.randint(a, b) to generate a random integer between a and b (inclusive)
import numpy as np
np.random.randint(0, 5)

4

In [212]:
# Try to create the function on your own.
def create_environment(num_rows, num_cols, num_obstacles):

    # 1. Create an empty list
    grid = []

    # 2. Fill the 2D list with "_".
    # Now we have an empty environment with specified size
    for i in range(num_rows):
        row = ["_"] * num_cols
        grid.append(row)

    # 3. Add obstacles randomly to the 2D list (Generate random row, col indices)
    count = 0 # number of obstacles
    while count < num_obstacles:
        row = np.random.randint(0, num_rows)
        col = np.random.randint(0, num_cols)
        # Don't overwrite the origin or the target
        if grid[row][col] == "_":
            grid[row][col] = "X"
            count += 1

    # 4. Add origin and target randomly
    # random orgin position
    while True:
      row = np.random.randint(0, num_rows)
      col = np.random.randint(0, num_cols)
      if grid[row][col] == "_":
          grid[row][col] = "O"
          break

    # random target position
    while True:
      row = np.random.randint(0, num_rows)
      col = np.random.randint(0, num_cols)
      if grid[row][col] == "_":
          grid[row][col] = "T"
          break

    # 5. Return the 2D list
    return grid

In [213]:
# Use * to quickly replicate elements in a list.
[1, 2, 3] * 3

[1, 2, 3, 1, 2, 3, 1, 2, 3]

In [214]:
# Test the function
grid = create_environment(5, 7, 10)
print_grid(grid)

X  _  _  _  _  X  T  
X  _  X  _  _  _  X  
X  X  X  _  _  _  _  
_  _  X  _  _  _  O  
_  _  _  _  X  _  _  


### Task 2: Agent Class

Create a class `Agent` that represents the intelligent agent.

In [220]:
class Agent:

    def __init__(self, grid):
        """
        Initialize important class variables
        """
        self.grid = grid
        self.num_rows = len(grid)
        self.num_cols = len(grid[0])

        for i in range(self.num_rows):
          for j in range(self.num_cols):
            # self.grid has data, num_rows and num_cols are counts
            if self.grid[i][j] == 'O':
              self.origin = [i,j]
            if self.grid[i][j] == 'T':
              self.target = [i,j]

        self.position = self.origin
        self.visited = [self.origin]
        self.path = [self.origin] # We will treat this as a stack

    def perceive_environment(self):
        """
        This method returns information of the up, down, left, and right cells adjacent to the agent.
        """
        row, col = self.position
        # Examine the cell above
        if row == 0:
            up = "Wall"
        elif self.grid[row-1][col] == "X":
            up = "Obstacle"
        else:
            up = "Empty"

        # Examine the cell below
        if row == self.num_rows - 1:
            down = "Wall"
        elif self.grid[row+1][col] == "X":
            down = "Obstacle"
        else:
            down = "Empty"

        # Examine left
        if col == 0:
            left = "Wall"
        elif self.grid[row][col-1] == "X":
            left = "Obstacle"
        else:
            left = "Empty"

        # Examine right
        if col == self.num_cols - 1:
            right = "Wall"
        elif self.grid[row][col+1] == "X":
            right = "Obstacle"
        else:
            right = "Empty"

        return up, down, left, right

    def decide_action(self):
        """
        Implement a decision-making process for the agent.
        """
        up, down, left, right = self.perceive_environment()
        row, col = self.position

        moves = []

        if right == "Empty" and ([row, col+1] not in self.visited):
            distance = abs(row - self.target[0]) + abs((col+1) - self.target[1])
            moves.append(("Right", distance))

        if down == "Empty" and ([row+1, col] not in self.visited):
            distance = abs((row+1) - self.target[0]) + abs(col - self.target[1])
            moves.append(("Down", distance))

        if left == "Empty" and ([row, col-1] not in self.visited):
            distance = abs(row - self.target[0]) + abs((col-1) - self.target[1])
            moves.append(("Left", distance))

        if up == "Empty" and ([row-1, col] not in self.visited):
            distance = abs((row-1) - self.target[0]) + abs(col - self.target[1])
            moves.append(("Up", distance))


        if len(moves) == 0:
            return "BackTrack"

        # pick the move with the smallest distance
        best_move = moves[0]
        for m in moves:
            if m[1] < best_move[1]:
                best_move = m

        return best_move[0]


    def take_action(self):
        """
        Update self.position, and check if the agent has reached the target.
        """
        action = self.decide_action()
        row, col = self.position # current position

        if action == "Right":
            self.position = [row, col+1]
        elif action == "Left":
            self.position = [row, col-1]
        elif action == "Up":
            self.position = [row-1, col]
        elif action == "Down":
            self.position = [row+1, col]
        elif action == "BackTrack":
            self.position = self.path[-2]
            self.path.pop() # remove the last position

        # Include the new position to self.path unless the action is
        # "BackTrack"
        if action != "BackTrack":
            self.path.append(self.position)
            self.visited.append(self.position)

        # if self.position not in self.visited:
        if self.position == self.target:
            print("Target is reached!")


In [216]:
list1 = [1, 2, 3, 4]
(3 not in list1)
# list1.pop()
# list1

False

In [217]:
# Test the correctness of the class
agent = Agent(grid)
print("Initial position:", agent.position)
for i in range(10):
    print("Action:", agent.decide_action())
    agent.take_action()
    print("Position:", agent.position)
print("Path:", agent.path)
print("Visited cells:", agent.visited)

Initial position: [3, 6]
Action: Up
Position: [2, 6]
Action: Left
Position: [2, 5]
Action: Up
Position: [1, 5]
Action: Left
Position: [1, 4]
Action: Up
Position: [0, 4]
Action: Left
Position: [0, 3]
Action: Down
Position: [1, 3]
Action: Down
Position: [2, 3]
Action: Right
Position: [2, 4]
Action: Down
Position: [3, 4]
Path: [[3, 6], [2, 6], [2, 5], [1, 5], [1, 4], [0, 4], [0, 3], [1, 3], [2, 3], [2, 4], [3, 4]]
Visited cells: [[3, 6], [2, 6], [2, 5], [1, 5], [1, 4], [0, 4], [0, 3], [1, 3], [2, 3], [2, 4], [3, 4]]


### Task 3: Agent Evaluation

Create a 7x7 grid with 7 obstacles and an instance of the `Agent` class. Simulate the agent's navigation and display the grid at each step to visualize the agent's progress. Use a `for` loop to keep the agent moving until it reaches the target or has taken 100 moves.

In [218]:
from copy import deepcopy
def display_agent(agent):
    """
    Display the agent in the grid
    """
    print("-"*20)
    grid = deepcopy(agent.grid)
    row, col = agent.position
    grid[row][col] = "A"
    print_grid(grid)

In [219]:
from IPython.display import clear_output # This function can clear the ouput of the cell

# Test the path-finding agent.
max_steps = 100
test_grid = create_environment(7, 7, 14)
# test_grid =  [  ["O", "_", "_", "_", "_"],
#              ["_", "X", "_", "X", "_"],
#              ["_", "X", "_", "X", "X"],
#              ["_", "X", "_", "_", "_"],
#              ["_", "X", "_", "X", "_"],
#              ["_", "X", "_", "X", "_"],
#              ["_", "X", "_", "X", "T"]]
# test_grid = [["O", "_", "_", "_", "_", "X", "_"],
#             ["_", "_", "_", "_", "X", "_", "_"],
#             ["_", "X", "_", "X", "_", "_", "X"],
#             ["X", "X", "_", "_", "_", "_", "X"],
#             ["_", "_", "_", "X", "_", "X", "X"],
#             ["_", "X", "_", "_", "_", "_", "_"],
#             ["_", "_", "X", "X", "_", "_", "T"]]
# test_grid = [["0","_","_","X","_","_"],
#             ["_","_","_","X","_","_"],
#             ["_","_","X","X","_","_"],
#             ["_","_","_","_","_","_"],
#             ["_","_","_","X","_","T"],
#             ]

test_agent = Agent(test_grid)
prev_position = test_agent.position
for step in range(max_steps):
    # clear_output()
    print("Action:", test_agent.decide_action())
    test_agent.take_action()
    display_agent(test_agent)
    print("Position:", test_agent.position)
    print("Visited nodes:", test_agent.visited)
    print("Path:", test_agent.path)

    # Terminate if the agent reaches the target
    if test_agent.position[0] == test_agent.target[0] \
        and test_agent.position[1] == test_agent.target[1]:
        print("Target is reached.")
        break

    # Terminate if the agent gets stuck
    if test_agent.position == prev_position:
        print("The agent is stuck.")
        break

    prev_position = test_agent.position.copy()

    # input("Step %d. Press Enter to continue..." % (step+1))

Action: Down
--------------------
_  _  _  _  _  _  _  
O  X  _  T  X  _  X  
A  X  X  _  X  _  X  
X  X  _  X  X  _  _  
_  X  _  _  _  _  _  
_  _  _  _  _  _  _  
X  _  _  _  X  _  _  
Position: [2, 0]
Visited nodes: [[1, 0], [2, 0]]
Path: [[1, 0], [2, 0]]
Action: BackTrack
--------------------
_  _  _  _  _  _  _  
A  X  _  T  X  _  X  
_  X  X  _  X  _  X  
X  X  _  X  X  _  _  
_  X  _  _  _  _  _  
_  _  _  _  _  _  _  
X  _  _  _  X  _  _  
Position: [1, 0]
Visited nodes: [[1, 0], [2, 0]]
Path: [[1, 0]]
Action: Up
--------------------
A  _  _  _  _  _  _  
O  X  _  T  X  _  X  
_  X  X  _  X  _  X  
X  X  _  X  X  _  _  
_  X  _  _  _  _  _  
_  _  _  _  _  _  _  
X  _  _  _  X  _  _  
Position: [0, 0]
Visited nodes: [[1, 0], [2, 0], [0, 0]]
Path: [[1, 0], [0, 0]]
Action: Right
--------------------
_  A  _  _  _  _  _  
O  X  _  T  X  _  X  
_  X  X  _  X  _  X  
X  X  _  X  X  _  _  
_  X  _  _  _  _  _  
_  _  _  _  _  _  _  
X  _  _  _  X  _  _  
Position: [0, 1]
Visited nod